# Usage Overview
The following is mocked example usage for `harmony-py`. 


In [2]:
import datetime
from shapely.geometry import Polygon
from harmony_py_mock import Action, Authenticate, Format, Request, Results
been  Synchronuous.

In [ ]:
# collection object CMR interaction TBD.
collection = Collection(id='C1940468263-POCLOUD')

# See more authentication options below. Creates a session object to be passed into 'submit()'.
auth = Authenticate()

# Individual parameters are validated when set.
req = Request(
    collection=collection,
    spatial=Bbox(lat=(40, 42), lon=(-107, -105)),  # could accept bbox, shapely, geojson (polygon)
    temporal=Temporal(start=datetime.date(2020, 6, 1), stop=datetime.date(2020, 6, 30)),
    format=Format.ZARR
)

# Not all keyword args need be supplied at once. Also, parameters may be replaced.
req.spatial = Polygon([(0, 0), (1, 1), (1, 0)])

# Optional; 'Request' action defaults to 'SUBSET'. Could also have specified during request instantiation.
req.action = Action.SUBSET

# A user may explicitly validate. Validation implicitly happens on 'submit()'.
Request.validate(req)

# Send request to Harmony and begin processing. Returns a 'Results'.
res = Request.submit(req, auth)



# Further Examples

In [ ]:
# Notes:
# I haven't seen a common bounding box class used in other software. It's simple enough that a namedtuple w/ validation should be sufficient
# The Builder Pattern as used in javascript thing().setSomething('asdf').setSomethingElse('asdf2') is not very Pythonic. Prefer kwargs instead. 
# As noted above, paramters validated on every 'Set', to the extent it's possible.
# Not all parameters need be provided at once.
# Request is validated upon submit()
# Need to research CMR library capabilities and plans
# (more notes responses coming)

## Request Parameters

## Authentication

In [ ]:
# Authentication options:
#   .) specify `username` and `password`
#   .) specify `username` and receive password prompt
#   .) specify .netrc
#   .) read .netrc in default location
#   .) read .env file
#   .) read ENV vars    

auth = Authenticate(username='myusername', password='supersecret')
# or
auth = Authenticate(username='myusername')
# or
auth = Authenticate(netrc='/usr/local/etc/some/path/.netrc')

# or
auth = Authenticate()


## Determine Service Availability and Variables / Working with CMR

## Async vs. Sync Request Submit()

## Retrieve Results in Cloud: In / Out of Region; Internet Retrieval

## Error Notification and Handling